In [7]:
import os
import numpy as np
import rasterio

RAW_DIR = "../data/raw_modis"
NDVI_OUT = "../data/ndvi_daily"

os.makedirs(NDVI_OUT, exist_ok=True)


In [8]:
def compute_ndvi(red, nir):
    red = red.astype("float32") * 0.0001
    nir = nir.astype("float32") * 0.0001
    return (nir - red) / (nir + red + 1e-6)


In [9]:
day_folders = sorted([
    d for d in os.listdir(RAW_DIR)
    if d.startswith("day_")
])

for day in day_folders:
    day_path = os.path.join(RAW_DIR, day)

    files = os.listdir(day_path)

    red_file = [f for f in files if "sur_refl_b01" in f][0]
    nir_file = [f for f in files if "sur_refl_b02" in f][0]

    red_path = os.path.join(day_path, red_file)
    nir_path = os.path.join(day_path, nir_file)

    with rasterio.open(red_path) as rsrc, rasterio.open(nir_path) as nsrc:
        red = rsrc.read(1)
        nir = nsrc.read(1)
        meta = rsrc.meta.copy()

    ndvi = compute_ndvi(red, nir)

    meta.update(dtype=rasterio.float32, count=1)

    out_file = f"ndvi_{day}.tif"
    out_path = os.path.join(NDVI_OUT, out_file)

    with rasterio.open(out_path, "w", **meta) as dst:
        dst.write(ndvi.astype("float32"), 1)

    print(f"Saved {out_file}")


Saved ndvi_day_01.tif
Saved ndvi_day_02.tif
Saved ndvi_day_03.tif
Saved ndvi_day_04.tif
Saved ndvi_day_05.tif
Saved ndvi_day_06.tif
Saved ndvi_day_07.tif
Saved ndvi_day_08.tif
Saved ndvi_day_09.tif
Saved ndvi_day_10.tif
Saved ndvi_day_11.tif
Saved ndvi_day_12.tif
Saved ndvi_day_13.tif
Saved ndvi_day_14.tif


In [10]:
import os
import numpy as np
import rasterio

NDVI_DIR = "../data/ndvi_daily"


ndvi_files = sorted([
    f for f in os.listdir(NDVI_DIR)
    if f.endswith(".tif")
])

print("NDVI files:")
for f in ndvi_files:
    print(f)


NDVI files:
ndvi_day_01.tif
ndvi_day_02.tif
ndvi_day_03.tif
ndvi_day_04.tif
ndvi_day_05.tif
ndvi_day_06.tif
ndvi_day_07.tif
ndvi_day_08.tif
ndvi_day_09.tif
ndvi_day_10.tif
ndvi_day_11.tif
ndvi_day_12.tif
ndvi_day_13.tif
ndvi_day_14.tif


In [11]:
ndvi_stack = []

for f in ndvi_files:
    path = os.path.join(NDVI_DIR, f)
    with rasterio.open(path) as src:
        ndvi = src.read(1)
    ndvi_stack.append(ndvi)

ndvi_stack = np.array(ndvi_stack)

print("Stack shape (T, H, W):", ndvi_stack.shape)


Stack shape (T, H, W): (14, 10342, 12242)


In [12]:
STACK_PATH = "../data/ndvi_stack.npy"

np.save(STACK_PATH, ndvi_stack)
print("NDVI stack saved at:", STACK_PATH)


NDVI stack saved at: ../data/ndvi_stack.npy


In [1]:
import cdsapi

c = cdsapi.Client()
print("CDS API connected successfully")


2025-12-19 20:02:44,795 INFO [2025-12-03T00:00:00Z] To improve our C3S service, we need to hear from you! Please complete this very short [survey](https://confluence.ecmwf.int/x/E7uBEQ/). Thank you.


CDS API connected successfully


In [4]:
import cdsapi
import os

# Output directory
ERA5_DIR = "../data/era5_raw"
os.makedirs(ERA5_DIR, exist_ok=True)

c = cdsapi.Client()

c.retrieve(
    "reanalysis-era5-single-levels",
    {
        "product_type": "reanalysis",
        "variable": [
            "10m_u_component_of_wind",
            "10m_v_component_of_wind",
            "relative_humidity"
        ],
        "year": "2023",
        "month": "01",
        "day": [
            "01", "02", "03", "04", "05", "06", "07",
            "08", "09", "10", "11", "12", "13", "14"
        ],
        "time": [
            "00:00", "01:00", "02:00", "03:00",
            "04:00", "05:00", "06:00", "07:00",
            "08:00", "09:00", "10:00", "11:00",
            "12:00", "13:00", "14:00", "15:00",
            "16:00", "17:00", "18:00", "19:00",
            "20:00", "21:00", "22:00", "23:00"
        ],
        "format": "netcdf"
    },
    os.path.join(ERA5_DIR, "era5_weather_20230101_20230114.nc")
)

print("ERA5 weather data download completed")


2025-12-19 22:32:44,632 INFO [2025-12-03T00:00:00Z] To improve our C3S service, we need to hear from you! Please complete this very short [survey](https://confluence.ecmwf.int/x/E7uBEQ/). Thank you.
2025-12-19 22:32:46,418 INFO [2025-12-11T00:00:00] Please note that a dedicated catalogue entry for this dataset, post-processed and stored in Analysis Ready Cloud Optimized (ARCO) format (Zarr), is available for optimised time-series retrievals (i.e. for retrieving data from selected variables for a single point over an extended period of time in an efficient way). You can discover it [here](https://cds.climate.copernicus.eu/datasets/reanalysis-era5-single-levels-timeseries?tab=overview)
2025-12-19 22:32:46,422 INFO Request ID is b960e9f8-e2b8-4078-97d0-5c4d98fbba98
2025-12-19 22:32:46,784 INFO status has been updated to accepted
2025-12-19 22:32:57,738 INFO status has been updated to running
2025-12-19 22:34:45,481 INFO status has been updated to successful


1e8df3e89e60632559d097a2da311b85.nc:   0%|          | 0.00/1.28G [00:00<?, ?B/s]

Recovering from connection error [('Connection broken: IncompleteRead(30712832 bytes read, 1343336655 more expected)', IncompleteRead(30712832 bytes read, 1343336655 more expected))], attempt 1 of 500
Retrying in 120 seconds


1e8df3e89e60632559d097a2da311b85.nc:   2%|2         | 29.0M/1.28G [00:00<?, ?B/s]

ERA5 weather data download completed


In [9]:
import xarray as xr

era5_path = "../data/era5_raw/era5_weather_20230101_20230114.nc"

ds = xr.open_dataset(era5_path)
ds




<xarray.Dataset> Size: 3GB
Dimensions:     (valid_time: 336, latitude: 721, longitude: 1440)
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 3kB 2023-01-01 ... 2023-01-14T23:...
  * latitude    (latitude) float64 6kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude   (longitude) float64 12kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
    number      int64 8B ...
    expver      (valid_time) <U4 5kB ...
Data variables:
    u10         (valid_time, latitude, longitude) float32 1GB ...
    v10         (valid_time, latitude, longitude) float32 1GB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-12-19T17:03 GRIB to CDM+CF via cfgrib-0.9.1...

In [11]:
print(ds.coords)


Coordinates:
  * valid_time  (valid_time) datetime64[ns] 3kB 2023-01-01 ... 2023-01-14T23:...
  * latitude    (latitude) float64 6kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude   (longitude) float64 12kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
    number      int64 8B ...
    expver      (valid_time) <U4 5kB ...


In [12]:
# Rename valid_time to time
ds = ds.rename({"valid_time": "time"})

print(ds.coords)


Coordinates:
  * time       (time) datetime64[ns] 3kB 2023-01-01 ... 2023-01-14T23:00:00
  * latitude   (latitude) float64 6kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude  (longitude) float64 12kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
    number     int64 8B ...
    expver     (time) <U4 5kB ...


In [13]:
ds_daily = ds.resample(time="1D").mean()
ds_daily


<xarray.Dataset> Size: 116MB
Dimensions:    (time: 14, latitude: 721, longitude: 1440)
Coordinates:
  * time       (time) datetime64[ns] 112B 2023-01-01 2023-01-02 ... 2023-01-14
  * latitude   (latitude) float64 6kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude  (longitude) float64 12kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
    number     int64 8B 0
Data variables:
    u10        (time, latitude, longitude) float32 58MB 2.37 2.37 ... 4.347
    v10        (time, latitude, longitude) float32 58MB 4.711 4.711 ... 0.6825
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-12-19T17:03 GRIB to CDM+CF via cfgrib-0.9.1...

In [14]:
import numpy as np

u10 = ds_daily["u10"]
v10 = ds_daily["v10"]

ds_daily["wind_speed"] = np.sqrt(u10**2 + v10**2)

ds_daily


<xarray.Dataset> Size: 174MB
Dimensions:     (time: 14, latitude: 721, longitude: 1440)
Coordinates:
  * time        (time) datetime64[ns] 112B 2023-01-01 2023-01-02 ... 2023-01-14
  * latitude    (latitude) float64 6kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude   (longitude) float64 12kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
    number      int64 8B 0
Data variables:
    u10         (time, latitude, longitude) float32 58MB 2.37 2.37 ... 4.347
    v10         (time, latitude, longitude) float32 58MB 4.711 4.711 ... 0.6825
    wind_speed  (time, latitude, longitude) float32 58MB 5.273 5.273 ... 4.4 4.4
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-12-19T17:03 GRIB to CDM+CF via cfgrib-0.9.1...

In [17]:
import cdsapi
import os

ERA5_DIR = "../data/era5_raw"
os.makedirs(ERA5_DIR, exist_ok=True)

c = cdsapi.Client()

c.retrieve(
    "reanalysis-era5-single-levels",
    {
        "product_type": "reanalysis",
        "variable": [
            "10m_u_component_of_wind",
            "10m_v_component_of_wind",
            "2m_temperature",
            "2m_dewpoint_temperature"
        ],
        "year": "2023",
        "month": "01",
        "day": [
            "01","02","03","04","05","06","07",
            "08","09","10","11","12","13","14"
        ],
        "time": [
            "00:00","01:00","02:00","03:00","04:00","05:00",
            "06:00","07:00","08:00","09:00","10:00","11:00",
            "12:00","13:00","14:00","15:00","16:00","17:00",
            "18:00","19:00","20:00","21:00","22:00","23:00"
        ],
        "format": "netcdf"
    },
    os.path.join(ERA5_DIR, "era5_weather_20230101_20230114_v2.nc")
)

print("ERA5 weather data (u10, v10, t2m, d2m) downloaded")


2025-12-19 23:55:11,013 INFO [2025-12-03T00:00:00Z] To improve our C3S service, we need to hear from you! Please complete this very short [survey](https://confluence.ecmwf.int/x/E7uBEQ/). Thank you.
2025-12-19 23:55:12,229 INFO [2025-12-11T00:00:00] Please note that a dedicated catalogue entry for this dataset, post-processed and stored in Analysis Ready Cloud Optimized (ARCO) format (Zarr), is available for optimised time-series retrievals (i.e. for retrieving data from selected variables for a single point over an extended period of time in an efficient way). You can discover it [here](https://cds.climate.copernicus.eu/datasets/reanalysis-era5-single-levels-timeseries?tab=overview)
2025-12-19 23:55:12,230 INFO Request ID is d8d86d57-4173-4c9d-ae60-b51c7c7ed3f3
2025-12-19 23:55:12,946 INFO status has been updated to accepted
2025-12-19 23:55:27,999 INFO status has been updated to successful


176a47c4c2b7befca4f88f18d4532f20.nc:   0%|          | 0.00/2.23G [00:00<?, ?B/s]

KeyboardInterrupt: 

In [19]:
import xarray as xr
import numpy as np


In [21]:
import xarray as xr

era5_path = "../data/era5_raw/era5_weather_20230101_20230114.nc"

ds = xr.open_dataset(era5_path)
ds


<xarray.Dataset> Size: 3GB
Dimensions:     (valid_time: 336, latitude: 721, longitude: 1440)
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 3kB 2023-01-01 ... 2023-01-14T23:...
  * latitude    (latitude) float64 6kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude   (longitude) float64 12kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
    number      int64 8B ...
    expver      (valid_time) <U4 5kB ...
Data variables:
    u10         (valid_time, latitude, longitude) float32 1GB ...
    v10         (valid_time, latitude, longitude) float32 1GB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-12-19T17:03 GRIB to CDM+CF via cfgrib-0.9.1...

In [22]:
import xarray as xr
import numpy as np

era5_path = "../data/era5_raw/era5_weather_20230101_20230114.nc"

ds = xr.open_dataset(era5_path)
ds


<xarray.Dataset> Size: 3GB
Dimensions:     (valid_time: 336, latitude: 721, longitude: 1440)
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 3kB 2023-01-01 ... 2023-01-14T23:...
  * latitude    (latitude) float64 6kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude   (longitude) float64 12kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
    number      int64 8B ...
    expver      (valid_time) <U4 5kB ...
Data variables:
    u10         (valid_time, latitude, longitude) float32 1GB ...
    v10         (valid_time, latitude, longitude) float32 1GB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-12-19T17:03 GRIB to CDM+CF via cfgrib-0.9.1...

In [23]:
ds = ds.rename({"valid_time": "time"})
print(ds.coords)


Coordinates:
  * time       (time) datetime64[ns] 3kB 2023-01-01 ... 2023-01-14T23:00:00
  * latitude   (latitude) float64 6kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude  (longitude) float64 12kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
    number     int64 8B ...
    expver     (time) <U4 5kB ...


In [24]:
ds_daily = ds.resample(time="1D").mean()
ds_daily


<xarray.Dataset> Size: 116MB
Dimensions:    (time: 14, latitude: 721, longitude: 1440)
Coordinates:
  * time       (time) datetime64[ns] 112B 2023-01-01 2023-01-02 ... 2023-01-14
  * latitude   (latitude) float64 6kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude  (longitude) float64 12kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
    number     int64 8B 0
Data variables:
    u10        (time, latitude, longitude) float32 58MB 2.37 2.37 ... 4.347
    v10        (time, latitude, longitude) float32 58MB 4.711 4.711 ... 0.6825
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-12-19T17:03 GRIB to CDM+CF via cfgrib-0.9.1...

In [25]:
ds_daily["wind_speed"] = np.sqrt(
    ds_daily["u10"]**2 + ds_daily["v10"]**2
)


In [26]:
# Convert Kelvin → Celsius
T = ds_daily["t2m"] - 273.15
Td = ds_daily["d2m"] - 273.15

# Saturation vapor pressure
es_T = 6.112 * np.exp((17.67 * T) / (T + 243.5))
es_Td = 6.112 * np.exp((17.67 * Td) / (Td + 243.5))

RH = 100 * (es_Td / es_T)

ds_daily["relative_humidity"] = RH
ds_daily


KeyError: "No variable named 't2m'. Variables on the dataset include ['u10', 'v10', 'number', 'latitude', 'longitude', 'time', 'wind_speed']"

In [27]:
import cdsapi
import os

ERA5_DIR = "../data/era5_raw"
os.makedirs(ERA5_DIR, exist_ok=True)

c = cdsapi.Client()

c.retrieve(
    "reanalysis-era5-single-levels",
    {
        "product_type": "reanalysis",
        "variable": [
            "10m_u_component_of_wind",
            "10m_v_component_of_wind",
            "2m_temperature",
            "2m_dewpoint_temperature"
        ],
        "year": "2023",
        "month": "01",
        "day": [
            "01","02","03","04","05","06","07",
            "08","09","10","11","12","13","14"
        ],
        "time": [
            "00:00","06:00","12:00","18:00"
        ],
        "area": [
            25, 70,   # North, West
            5, 90     # South, East  (South India)
        ],
        "format": "netcdf"
    },
    os.path.join(ERA5_DIR, "era5_weather_si_20230101_20230114.nc")
)

print("ERA5 South India weather data downloaded successfully")


2025-12-20 15:36:58,063 INFO [2025-12-03T00:00:00Z] To improve our C3S service, we need to hear from you! Please complete this very short [survey](https://confluence.ecmwf.int/x/E7uBEQ/). Thank you.
2025-12-20 15:36:59,231 INFO [2025-12-11T00:00:00] Please note that a dedicated catalogue entry for this dataset, post-processed and stored in Analysis Ready Cloud Optimized (ARCO) format (Zarr), is available for optimised time-series retrievals (i.e. for retrieving data from selected variables for a single point over an extended period of time in an efficient way). You can discover it [here](https://cds.climate.copernicus.eu/datasets/reanalysis-era5-single-levels-timeseries?tab=overview)
2025-12-20 15:36:59,233 INFO Request ID is 7b746c9a-c8b5-489b-8a95-9e071d216ccd
2025-12-20 15:36:59,613 INFO status has been updated to accepted
2025-12-20 15:37:34,199 INFO status has been updated to successful


af677428b5d803fbf66628275e6742ed.nc:   0%|          | 0.00/2.70M [00:00<?, ?B/s]

ERA5 South India weather data downloaded successfully


In [28]:
import xarray as xr
import numpy as np

era5_path = "../data/era5_raw/era5_weather_si_20230101_20230114.nc"

ds = xr.open_dataset(era5_path)
ds


<xarray.Dataset> Size: 6MB
Dimensions:     (valid_time: 56, latitude: 81, longitude: 81)
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 448B 2023-01-01 ... 2023-01-14T18...
  * latitude    (latitude) float64 648B 25.0 24.75 24.5 24.25 ... 5.5 5.25 5.0
  * longitude   (longitude) float64 648B 70.0 70.25 70.5 ... 89.5 89.75 90.0
    number      int64 8B ...
    expver      (valid_time) <U4 896B ...
Data variables:
    u10         (valid_time, latitude, longitude) float32 1MB ...
    v10         (valid_time, latitude, longitude) float32 1MB ...
    t2m         (valid_time, latitude, longitude) float32 1MB ...
    d2m         (valid_time, latitude, longitude) float32 1MB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-12-20T10:07 GRIB to CDM+CF via cfgrib-0.9.1...

In [29]:
ds = ds.rename({"valid_time": "time"})


In [30]:
ds_daily = ds.resample(time="1D").mean()


In [31]:
ds_daily["wind_speed"] = np.sqrt(
    ds_daily["u10"]**2 + ds_daily["v10"]**2
)


In [32]:
T = ds_daily["t2m"] - 273.15
Td = ds_daily["d2m"] - 273.15

es_T = 6.112 * np.exp((17.67 * T) / (T + 243.5))
es_Td = 6.112 * np.exp((17.67 * Td) / (Td + 243.5))

ds_daily["relative_humidity"] = 100 * (es_Td / es_T)


In [33]:
wind_np = ds_daily["wind_speed"].values
rh_np = ds_daily["relative_humidity"].values

print(wind_np.shape, rh_np.shape)


(14, 81, 81) (14, 81, 81)
